In [38]:
#Imports

from datetime import datetime, timedelta
import pytz
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [39]:
#Weather Function

def get_weather(city):
    geocoding_url = f'https://geocoding-api.open-meteo.com/v1/search?name={city}'
    geocoding_response = requests.get(geocoding_url)
    if geocoding_response.status_code != 200 or not geocoding_response.json().get('results'):
        return None, None

    location = geocoding_response.json()['results'][0]
    latitude, longitude = location['latitude'], location['longitude']
    timezone_name = location['timezone']

    weather_url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true'
    weather_response = requests.get(weather_url)
    if weather_response.status_code != 200:
        return None, None

    return weather_response.json(), timezone_name

In [40]:
#Display Weather Function

def display_weather(data, city, timezone_name):
    if not data:
        print("City not found or error fetching data.")
        return

    weather = data['current_weather']
    temperature, wind_speed, weather_code = weather['temperature'], weather['windspeed'], weather['weathercode']
    utc_time_str = weather['time']

    try:
        utc_time = datetime.fromisoformat(utc_time_str.replace('Z', '+00:00'))
        local_timezone = pytz.timezone(timezone_name)
        local_time = utc_time.astimezone(local_timezone) + timedelta(minutes=10)

        print(f"Temperature: {temperature}°C")
        print(f"Local Time in {city}: {local_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Weather: {weather_code}")
        print(f"Wind Speed: {wind_speed} km/h")
    except Exception as e:
        print(f"Error converting time: {e}")

In [41]:
#Visualize Weather Function

def visualize_weather(data):
    if not data:
        return

    temperature = data['current_weather']['temperature']
    color = 'lightblue' if temperature < 0 else 'green' if temperature < 15 else 'yellow' if temperature < 30 else 'red'

    plt.bar(['Current Temp'], [temperature], color=color)
    plt.title("Current Temperature")
    plt.ylabel('Temperature (°C)')
    plt.show()

In [ ]:
#Interactive Display Function and Widgets

def display_weather_interactive(city):
    data, timezone_name = get_weather(city)
    if data:
        display_weather(data, city, timezone_name)
        visualize_weather(data)
    else:
        print(f"Weather data for {city} not found.")

city_widget = widgets.Text(value='Enter a city name', placeholder='Enter a city name', description='City:')
button = widgets.Button(description="Get Weather")
output = widgets.Output()

def on_button_click(button):
    with output:
        output.clear_output()
        display_weather_interactive(city_widget.value)

button.on_click(on_button_click)
display(widgets.VBox([city_widget, button, output]))